In [1]:
import numpy as np
import pandas as pd
import os

#### Load data

In [2]:
data_path = "/workspace/lang-detect/txt/"
dir_list = os.listdir(data_path)
print(dir_list)

['sl', 'es', 'el', 'nl', 'hu', 'it', 'bg', 'sk', 'da', 'sv', 'cs', 'lt', 'de', 'en', 'pl', 'fr', 'fi', 'lv', 'pt', 'et', 'ro']


In [3]:
def read_data(filepath):
    with open(filepath, "r") as f:
        lines = f.readlines()
        if len(lines) > 1:
            return lines[1].strip("\n")
        return None

In [4]:
%%time
data, labels = [], []
for dir_name in dir_list:
    files_list = os.listdir(data_path + dir_name)
    for f in files_list:
        sent = read_data(data_path + dir_name + "/" + f)
        if sent:
            data.append(sent)
            labels.append(dir_name)

CPU times: user 2.81 s, sys: 1.69 s, total: 4.5 s
Wall time: 4.53 s


In [5]:
print("Length of data", len(data))

('Length of data', 186458)


In [6]:
# Check data sample
import random
rand_indices = random.sample(range(len(data)),  20)
for i in rand_indices:
    print(data[i],labels[i])

('10. Qualit\xc3\xa0 delle acque destinate alla molluschicoltura (versione codificata) (votazione) ', 'it')
('\xc3\x84\xc3\xa4nestykset', 'fi')
('4. Flughafenkapazit\xc3\xa4t und Bodenabfertigung: der Weg zu mehr Effizienz (Abstimmung) ', 'de')
('Istuntokauden keskeytt\xc3\xa4minen', 'fi')
('Durchf\xc3\xbchrungsma\xc3\x9fnahmen (Artikel 88 GO): siehe Protokoll', 'de')
('N\xc4\x81kamo s\xc4\x93\xc5\xbeu datumi (sk. protokolu)', 'lv')
('\xd0\x97\xd0\xb0\xd0\xba\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb5 \xd0\xbd\xd0\xb0 \xd0\xb7\xd0\xb0\xd1\x81\xd0\xb5\xd0\xb4\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5\xd1\x82\xd0\xbe', 'bg')
('Aprobaci\xc3\xb3n del Acta de la sesi\xc3\xb3n anterior', 'es')
('Zaklju\xc4\x8dek seje', 'sl')
('In het register ingeschreven schriftelijke verklaringen (artikel 116 van het reglement) ', 'nl')
('4. Globaliseerumisega Kohanemise Euroopa Fondi kasutuselev\xc3\xb5tmine: Heidelberger Druckmaschinen AG, Saksamaa (', 'et')
('3. Som\xc3\xa1lia (vota\xc3\xa7\xc3\xa3o) ', 'pt'

#### Preprocess the data

In [7]:
import re, string

pattern = re.compile("[\d{}]+$".format(re.escape(string.punctuation)))

def preprocess(text):
    # remove punctuations
    text = re.sub("[()/.:?,!@#$%\"']", "", text)
    new_string = ""
    for token in text.split():
        # check if no alphabet character
        if not pattern.match(token):
            # if not in all-caps, make it lower case
            if not token.isupper():
                token = token.lower()
            new_string += (token + " ")
    return new_string

test_text = "12. Aftale mellem EF\" 'og' Japans !2### ( 100.9 lin 1233 ??? p\xc3\xb6yt\xc3\xa4kirjan hyv\xc3\xa4ksyminen:"
preprocess(test_text)

'aftale mellem EF og japans lin p\xc3\xb6yt\xc3\xa4kirjan hyv\xc3\xa4ksyminen '

In [8]:
%%time
for i in range(len(data)):
    data[i] = preprocess(data[i])

CPU times: user 1.17 s, sys: 10.4 ms, total: 1.18 s
Wall time: 1.18 s


In [9]:
#test
print(data[:5])

['predlog splo\xc5\xa1nega prora\xc4\x8duna za leto oddelek III ', '\xc5\xbdenske in vodenje podjetij ', 'razmere na bli\xc5\xbenjem vzhodugaza glasovanje ', 'predlo\xc5\xbeitev dokumentov glej zapisnik ', 'javna ponudba vrednostnih papirjev in uskladitev zahtev v zvezi s preglednostjo razprava ']


#### Create dictionary for each language

In [10]:
from collections import defaultdict

lang_dict = defaultdict(lambda: defaultdict(int))

for i in range(len(data)):
    for token in data[i].split():
        if (labels[i] in lang_dict) and (token in lang_dict[labels[i]]):
            lang_dict[labels[i]][token] += 1
        else:
            lang_dict[labels[i]][token] = 0

In [11]:
lang_dict['en']['the']

5324

In [12]:
# Vocabulary size in each language
for key in lang_dict.keys():
    print(key, len(lang_dict[key].keys()))

('el', 6713)
('fr', 4738)
('bg', 5449)
('nl', 5100)
('ro', 5111)
('pt', 4241)
('lv', 6186)
('sv', 5645)
('de', 5617)
('it', 4819)
('hu', 6841)
('sk', 6480)
('et', 6733)
('lt', 6431)
('en', 4035)
('pl', 6485)
('sl', 6391)
('cs', 6002)
('fi', 7015)
('da', 5240)
('es', 4273)


#### Save the data

In [13]:
import json

with open('lang_dict.json', 'w') as outfile:
    json.dump(lang_dict, outfile)

In [14]:
# Test bg language data
count = 0
for i in range(len(data)):
    if labels[i] == 'bg':
        print(data[i])
        count += 1
    if count == 10:
        break

Доклад относно проекта на общ бюджет на Европейския съюз за финансовата година Раздел III Комисия 
Жените и ръководството на предприятия 
Положението в Средния изтокИвицата Газа гласуване 
Внасяне на документи вж протоколи 
Въпроси с искане за устен отговор и писмени декларации внасяне вж протокола 
Десета годишнина от приемането на Резолюция на Съвета за сигурност на ООН относно жените мира и сигурността внесени предложения за резолюция вж протокола 
Ред за работата на Парламента вж протоколи 
Поправки и намерения за гласуване вж протоколи 
Сътрудничество между съдилища на държавите-членки при събирането на доказателства по граждански или търговски дела кратко представяне 
Поправки член 204а от Правилника за дейността вж протокола 
